In [ ]:
#Here’s an example of how to apply a load profile to Load on Bus 1 and iterate over a few time steps:


In [12]:
import pandas as pd
import pandapower as pp
import numpy as np

# Initialize network
net = pp.create_empty_network()

In [13]:
# Create high voltage buses (11 kV)
bus1 = pp.create_bus(net, vn_kv=11, name="Bus 1")
bus2 = pp.create_bus(net, vn_kv=11, name="Bus 2")

In [14]:
# Create medium voltage buses (0.4 kV)
bus3 = pp.create_bus(net, vn_kv=0.4, name="Bus 3")
bus4 = pp.create_bus(net, vn_kv=0.4, name="Bus 4")

In [15]:
# Low voltage buses connected to loads
bus5 = pp.create_bus(net, vn_kv=0.4, name="Bus 5")
bus6 = pp.create_bus(net, vn_kv=0.4, name="Bus 6")

In [16]:
# External grid connections
pp.create_ext_grid(net, bus=bus1, vm_pu=1.0, name="Grid Connection 1")
pp.create_ext_grid(net, bus=bus2, vm_pu=1.0, name="Grid Connection 2")

1

In [17]:
# Transformers with 11 kV to 0.4 kV ratio
pp.create_transformer_from_parameters(
    net, hv_bus=bus1, lv_bus=bus3, sn_mva=1, vn_hv_kv=11, vn_lv_kv=0.4, 
    vk_percent=6, vkr_percent=0.5, pfe_kw=0, i0_percent=0.1, name="Transformer 1"
)
pp.create_transformer_from_parameters(
    net, hv_bus=bus2, lv_bus=bus4, sn_mva=1, vn_hv_kv=11, vn_lv_kv=0.4, 
    vk_percent=6, vkr_percent=0.5, pfe_kw=0, i0_percent=0.1, name="Transformer 2"
)

1

In [18]:
# High-voltage loads (connected to Bus 1 and Bus 2)
pp.create_load(net, bus=bus1, p_mw=0.8, q_mvar=0.4, name="HV Load 1")
pp.create_load(net, bus=bus2, p_mw=0.9, q_mvar=0.45, name="HV Load 2")

1

In [19]:
# Low-voltage loads (connected to Bus 5 and Bus 6)
pp.create_load(net, bus=bus5, p_mw=0.4, q_mvar=0.2, name="LV Load 3")
pp.create_load(net, bus=bus6, p_mw=0.3, q_mvar=0.15, name="LV Load 4")

3

In [20]:
# Lines between transformers and load buses
pp.create_line_from_parameters(
    net, from_bus=bus3, to_bus=bus5, length_km=0.1, r_ohm_per_km=0.1, 
    x_ohm_per_km=0.2, c_nf_per_km=0, max_i_ka=0.4, name="Line 1"
)
pp.create_line_from_parameters(
    net, from_bus=bus4, to_bus=bus6, length_km=0.1, r_ohm_per_km=0.1, 
    x_ohm_per_km=0.2, c_nf_per_km=0, max_i_ka=0.4, name="Line 2"
)

1

In [22]:
# Define a load profile for 'Load on Bus 1'
time_steps = 10  # Example: 10 time steps
load_profile_bus1 = np.linspace(0.5, 1.5, time_steps)  # Linearly increasing load (0.5 MW to 1.5 MW)

# Run load flow for each time step and apply the load profile
for t in range(time_steps):
    # Update load on Bus 1 with the profile value for this time step
    net.load.at[0, "p_mw"] = load_profile_bus1[t]  # Load on Bus 1

    # Run load flow
    pp.runpp(net,numba=False)
    
    # Print results for this time step
    print(f"Time step {t + 1}:")
    print("Load on Bus 1 (MW):", load_profile_bus1[t])
    print("Bus voltages (pu):\n", net.res_bus.vm_pu)
    print("Line loading (%):\n", net.res_line.loading_percent)
    print("\n" + "-"*40 + "\n")


Time step 1:
Load on Bus 1 (MW): 0.5
Bus voltages (pu):
 0    1.000000
1    1.000000
2    0.983657
3    0.988233
4    0.929008
5    0.948241
Name: vm_pu, dtype: float64
Line loading (%):
 0    173.706114
1    127.637119
Name: loading_percent, dtype: float64

----------------------------------------

Time step 2:
Load on Bus 1 (MW): 0.6111111111111112
Bus voltages (pu):
 0    1.000000
1    1.000000
2    0.983657
3    0.988233
4    0.929008
5    0.948241
Name: vm_pu, dtype: float64
Line loading (%):
 0    173.706114
1    127.637119
Name: loading_percent, dtype: float64

----------------------------------------

Time step 3:
Load on Bus 1 (MW): 0.7222222222222222
Bus voltages (pu):
 0    1.000000
1    1.000000
2    0.983657
3    0.988233
4    0.929008
5    0.948241
Name: vm_pu, dtype: float64
Line loading (%):
 0    173.706114
1    127.637119
Name: loading_percent, dtype: float64

----------------------------------------

Time step 4:
Load on Bus 1 (MW): 0.8333333333333333
Bus voltages (p

In [23]:
# Switch to change topology between Bus 5 and Bus 6
switch = pp.create_switch(net, bus=bus5, element=bus6, et="b", closed=True, name="Topology Switch",numba=False)

# Function to run load flow with switch open and closed
def run_loadflow(switch_closed):
    net.switch.loc[switch, 'closed'] = switch_closed
    pp.runpp(net,numba=False)
    status = "closed" if switch_closed else "open"
    print(f"\n=== Load Flow Results with Switch {status} ===")
    print("Bus Voltages:\n", net.res_bus[["vm_pu", "va_degree"]])
    print("Transformer Loading:\n", net.res_trafo[["p_hv_mw", "q_hv_mvar", "p_lv_mw", "q_lv_mvar"]])
    print("Line Loadings:\n", net.res_line[["loading_percent"]])
    print("Load Power:\n", net.res_load[["p_mw", "q_mvar"]])


In [24]:
# Run load flows
run_loadflow(switch_closed=True)  # Switch closed
run_loadflow(switch_closed=False) # Switch open


=== Load Flow Results with Switch closed ===
Bus Voltages:
       vm_pu  va_degree
0  1.000000   0.000000
1  1.000000   0.000000
2  0.985993  -1.196596
3  0.985993  -1.196596
4  0.938770  -3.228111
5  0.938770  -3.228111
Transformer Loading:
     p_hv_mw  q_hv_mvar   p_lv_mw  q_lv_mvar
0  0.361729   0.208106 -0.360859  -0.196719
1  0.361729   0.208106 -0.360859  -0.196719
Line Loadings:
    loading_percent
0       150.412282
1       150.412282
Load Power:
    p_mw  q_mvar
0   1.5    0.40
1   0.9    0.45
2   0.4    0.20
3   0.3    0.15

=== Load Flow Results with Switch open ===
Bus Voltages:
       vm_pu  va_degree
0  1.000000   0.000000
1  1.000000   0.000000
2  0.983657  -1.376851
3  0.988233  -1.018991
4  0.929008  -3.728719
5  0.948241  -2.738888
Transformer Loading:
     p_hv_mw  q_hv_mvar   p_lv_mw  q_lv_mvar
0  0.415643   0.243820 -0.414483  -0.228967
1  0.308446   0.174119 -0.307820  -0.165640
Line Loadings:
    loading_percent
0       173.706114
1       127.637119
Load Power: